# Preparation

## Library

In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.sampler import WeightedRandomSampler
from torch.optim import lr_scheduler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.inspection import permutation_importance

from fetchData import fetchdata, cv_events
import __MLP
# from __MLP import getSamplers, convert_df_to_unsqueezed_tensor, train_sequential, clf_report
import random

import __Preprocessing
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

pd.set_option('display.max_columns', None)

In [187]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [188]:
seed = 42
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)


torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

def _init_fn(worker_id):
    np.random.seed(int(seed))

def test_data_process(X_test, y_test):
    tensor_x1 = torch.Tensor(X_test.values).unsqueeze(1)
    tensor_y1 = torch.Tensor(y_test.values).unsqueeze(1)
    test_dataset = TensorDataset(tensor_x1,tensor_y1)

    batch_size = 8

    # train_sampler, test_sampler = __MLP.getSamplers(pheme_y, tensor_x2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    data = next(iter(test_dataloader))
    print("mean: %s, std: %s" %(data[0].mean(), data[0].std()))

    test_size = int(tensor_y1.size(0))

    print("Test Size",test_size)

    # predict_batch
    return test_dataloader, test_size


def test_data_process(X_test, y_test):

    tensor_x1 = torch.Tensor(X_test.values).unsqueeze(1)
    tensor_y1 = torch.Tensor(y_test.values).unsqueeze(1)
    test_dataset = TensorDataset(tensor_x1,tensor_y1)

    batch_size = 8

    # train_sampler, test_sampler = __MLP.getSamplers(pheme_y, tensor_x2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    data = next(iter(test_dataloader))
    print("mean: %s, std: %s" %(data[0].mean(), data[0].std()))

    test_size = int(tensor_y1.size(0))

    print("Test Size",test_size)

    # predict_batch
    return test_dataloader, test_size
def predict(model, criterion, val_dataloader, val_size):
    model.eval()
    val_label_list = []
    # val_preds_list = []
    running_val_preds = []

    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0
        f1_running = 0
        for j, val in enumerate(val_dataloader, 0):
            val_x, val_label = val
            val_x, val_label = val_x.float(), val_label.float()
            val_outputs = model(val_x)
            val_preds = val_outputs.squeeze(1) > 0.0
            f1_running += (f1_score(val_label, val_preds,zero_division=True) * val_x.size(0))
            v_loss = criterion(val_outputs, val_label.unsqueeze(1))
            val_loss += (v_loss.item() * val_x.size(0))
            val_corrects += torch.sum(val_preds == val_label)
            val_label_list.append(val_label)
            running_val_preds.append(val_preds)

    running_val_preds = torch.cat(running_val_preds, 0)
    val_label_list = torch.cat(val_label_list, 0)
    val_corrects = val_corrects
    val_loss = val_loss/val_size
    val_acc = val_corrects.double().numpy() / val_size
    f1_running /= val_size
    print("accuracy_score:\t\t%.4f" % val_acc)
    print('Precision Score:\t%.4f' % precision_score(val_label_list,running_val_preds))
    print('Recall Score:\t\t%.4f' % recall_score(val_label_list,running_val_preds))
    print("f1_score:\t\t%.4f" % f1_running)
    print("Test_loss:\t\t%.4f" % val_loss)


## Data

In [207]:
# Final
pheme_sparse_final = pd.read_csv('./data/_PHEME_sparse.csv')
pheme_y = pd.read_csv('./data/_PHEME_target.csv').target
pheme_pos_final = pd.read_csv('./data/_PHEME_postags.csv')
pheme_thread_final_avg = pd.read_csv('./data/_PHEME_thread_avg.csv')
pheme_thread_final_std = pd.read_csv('./data/_PHEME_thread_std.csv')

ext_pos_final = pd.read_csv('./data/_PHEMEext_postags.csv')
ext_sparse_final = pd.read_csv('./data/_PHEMEext_sparse.csv')
ext_y = pd.read_csv('./data/_PHEMEext_text.csv').target
ext_thread_final_avg = pd.read_csv('./data/_PHEMEext_thread_avg.csv')
ext_thread_final_std = pd.read_csv('./data/_PHEMEext_thread_std.csv')

pheme_bert_simple_normal = pd.read_csv('./data/_PHEME_Bert_final_simple_nrmzd.csv')
ext_bert_simple_normal = pd.read_csv('./data/_PHEMEext_Bert_final_simple_nrmzd.csv')

pheme_bert_brackets_normal = pd.read_csv('./data/_PHEME_Bert_final_brackets_nrmzd.csv')
ext_bert_brackets_normal = pd.read_csv('./data/_PHEMEext_Bert_final_brackets_nrmzd.csv')

pheme_event = pd.read_csv('./data/_PHEME_text.csv')['Event']
ext_event = pd.read_csv('./data/_PHEMEext_text.csv').Event
pheme_AVGw2v = pd.read_csv('./data/_PHEME_text_AVGw2v.csv').drop(['token'],axis=1)
ext_AVGw2v = pd.read_csv('./data/_PHEMEext_text_AVGw2v.csv').drop(['token'],axis=1)

In [6]:
print("pheme_sparse_final", pheme_sparse_final.shape)
print(pheme_sparse_final.columns, "\n")
print("pheme_pos_final", pheme_pos_final.shape)
x = zip(pheme_pos_final.columns.values, ['Interjection', 'Hashtag', 'coordinating conjunction', 'punctuation', 'at-mention', 'proper noun', 'adjective' 'determiner' 'other', 'nominal + verbal',
    'common noun', 'pronoun', 'pre/postposition/subordinating conjunction', 'adverb', 'nominal + possessive', 'verb particle', 'URL or email', 'verb incl. copula, auxiliaries',
 'existential there, predeterminers', 'proper noun + possessive', 'discourse marker'])
print(set(x), "\n")
print("pheme_thread_final", pheme_thread_final_avg.shape)
print(pheme_thread_final_avg.columns, "\n")
print("ext_bert_simple_normal", ext_bert_simple_normal.shape)
print(pheme_bert_simple_normal.columns, "\n")


pheme_sparse_final (5802, 28)
Index(['emoji_count', 'URLcount', 'has_media', 'Skepticism', 'MentionCount',
       'FirstPersonPronoun', 'SecondPersonPronoun', 'ThirdPersonPronoun',
       'test_auxiliary', 'test_tentat', 'test_certain', 'Numeral',
       'char_count', 'word_count', 'HashTag', 'has_question', 'has_exclaim',
       'has_period', 'capital_ratio', 'retweet_count', 'tweet_count',
       'listed_count', 'friends_count', 'follower_count', 'followers/friend',
       'favourites_count', 'account_age_days', 'verified'],
      dtype='object') 

pheme_pos_final (5802, 21)
{('s', 'URL or email'), ('&', 'coordinating conjunction'), ('o', 'adverb'), ('n', 'pre/postposition/subordinating conjunction'), ('p', 'nominal + possessive'), ('r', 'verb particle'), ('t', 'verb incl. copula, auxiliaries'), ('x', 'discourse marker'), ('g', 'common noun'), ('u', 'existential there, predeterminers'), ('a', 'adjectivedeterminerother'), ('@', 'at-mention'), ('d', 'nominal + verbal'), ('^', 'proper n

## Functions

In [7]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

seed = 42

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [8]:
def getDataSize(tensor_x1, tensor_y1, tensor_x2, tensor_y2):
    train_size = int(tensor_y1.size(0))
    test_size = int(tensor_y2.size(0))

    print("Variables)\n\tTrain:%s\n\tTest: %s"%(tensor_x1.size(),tensor_x2.size()))
    # print("\tTargets:%s \ %s"%(tensor_y1.size()[0],tensor_y2.size()[0]))
    print("Train Size",train_size,"Test Size",test_size)
    print()
    return train_size, test_size

In [9]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()


In [148]:
def cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epochs, verbose=True,scaling=False):
    # cv_pd_list[?][0]은 Training cv_pd_list[?][1] Testing
    cv_pd_list = []
    data = pd.concat([dataset, pheme_event, pheme_y], axis=1)
    NUM_EVENT = data.Event.unique().shape[0]
    EVENTS = data.Event.unique()
    results = {}

    for i, d in enumerate(EVENTS):
        df1, df2 = [x for _, x in data.groupby(data['Event'] != d)]
        df1.reset_index(inplace=True, drop=True)
        df2.reset_index(inplace=True, drop=True)
        cv_pd_list.append([df2, df1])

    # for train, test in cv_pd_list:
    #     print("Train: %s \ Test: %s" % (train.shape, test.shape))
    print()

    for index, fold in enumerate(cv_pd_list):

        # DATA PREPARATION
        train, test = fold
        print("FOLD %d\n----------------------------------------------------------------------------" % (int(index)+1))
        train_target = train.pop('target')
        train.pop('Event')
        test_target = test.pop('target')
        test.pop('Event')

        if scaling==True:
            scaler = StandardScaler()
            train = pd.DataFrame(scaler.fit_transform(train))
            test = pd.DataFrame(scaler.transform(test))

        tensor_x1, tensor_y1, tensor_x2, tensor_y2 = __MLP.convert_df_to_unsqueezed_tensor(
            train.values, train_target, test.values, test_target.values)
        train_dataset = TensorDataset(tensor_x1, tensor_y1)
        test_dataset = TensorDataset(tensor_x2, tensor_y2)

        batch_size = 16

        train_sampler, test_sampler = __MLP.getSamplers(train_target, tensor_x2)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                    sampler=train_sampler, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                                    shuffle=False, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)

        data = next(iter(train_dataloader))
        print("mean: %s, std: %s" % (data[0].mean(), data[0].std()))

        train_size, test_size = getDataSize(
            tensor_x1, tensor_y1, tensor_x2, tensor_y2)
        modelname = model.__class__.__name__

        # model = sparse_model()

        # model_sparse = sparse_model()
        # criterion = nn.BCEWithLogitsLoss()
        # optimizer = optim.SGD(model_sparse.parameters(), lr=0.01, momentum=0.9)
        # optimizer = optim.Adam(model_sparse.parameters(), lr=5e-5, eps=1e-8, weight_decay=1e-7)
        optimizer = AdamW(model.parameters(),
                        # lr=5e-5,    # Default learning rate
                        lr=5e-5,    # Default learning rate
                        eps=1e-8,    # Default epsilon value
                        weight_decay=1e-6
                        )

        total_steps = len(train_dataloader) * epochs
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=0,  # Default value
                                                    num_training_steps=total_steps)

        print(f'Model: {modelname}')
        print(f'Epochs: {epochs}')
        PATH = "./Model/"+modelname+"_"+str(index+1)+".pt"
        print(f'PATH: {PATH}\n')

        training_acc = []
        training_loss = []

        # train_acc, train_loss, val_acc, val_loss_list = __MLP.train_sequential(model=model, num_epochs=epochs, patience=patience, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_dataloader, train_size=train_size, test_loader=test_dataloader, test_size=test_size, PATH=PATH)

        # Run the training loop for defined number of epochs
        for epoch in range(0, epochs):

            # Print epoch
            if (verbose!=False):
                # pass
                print(f'Starting epoch {epoch+1}')
            elif (verbose!=True):
                if epoch%25 == 24:
                    print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            running_corrects = 0.0
            running_loss = 0.0

            # Iterate over the DataLoader for training data
            for i, data in enumerate(train_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Zero the gradients
                optimizer.zero_grad()

                # Perform forward pass
                outputs = model(inputs)

                outputs = outputs.view(outputs.size(0), -1)

                # Compute Prediction Outputs
                # preds = outputs.squeeze(1) > 0.0
                preds = outputs > 0.0

                # Compute loss
                loss = criterion(outputs, targets)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == targets.data).data

                # Perform backward pass
                loss.backward()

                # Perform optimization and Scheduler
                optimizer.step()
                scheduler.step()

                # Print statistics
                # current_loss += loss.item() # 원본
                # if i % len(train_dataloader) == len(train_dataloader)-1:
                #     print('Loss after mini-batch %5d: %.3f' %
                #           (i + 1, current_loss / i+1))

                current_loss += loss.item() * inputs.size(0)
                if verbose == True:
                    if i % len(train_dataloader) == len(train_dataloader)-1:
                        print("Loss/ACC after mini-batch %5d: %.3f / %.4f" %
                            (i + 1, current_loss / train_size, running_corrects/train_size))

            # epoch_acc = running_corrects.double() / train_size
            epoch_acc = running_corrects / train_size
            epoch_loss = running_loss / train_size
            training_acc.append(epoch_acc)
            training_loss.append(epoch_loss)
            # print('Epoch {}/{}\tTrain) Acc: {:.4f}, Loss: {:.4f}'.format(epoch+1,
                                                                        #  epochs, epoch_acc, epoch_loss))
        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('<Starting TESTING>')

        # Saving the model
        # save_path = f'./model-fold-{fold}.pth'
        torch.save(model.state_dict(), PATH)

        # Evaluation for this fold
        correct, total = 0, 0
        val_corrects=0
        f1_batch_epoch = 0
        val_label_list = []
        val_loss = 0
        with torch.no_grad():

            # Iterate over the test data and generate predictions
            for i, data in enumerate(test_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Generate outputs
                outputs = model(inputs)

                # Set total and correct
                outputs = outputs.view(outputs.size(0), -1).float()
                predicted = (outputs > 0.0).float()
                correct += (predicted == targets).float().sum().item()

                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                #!
                preds = (outputs > 0.0).float()
                # running_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == targets.data).data
                # f1_batch = f1_score(targets.cpu(), outputs.sigmoid().cpu() > 0.5, average='macro')
                f1_batch = f1_score(targets.cpu(), preds, average='macro')
                f1_batch_epoch += f1_batch * inputs.size(0)
                # f1_running += (f1_score(targets, preds, average='macro') * inputs.size(0))
                total += targets.size(0)
            
            if verbose==True:
                # print(f'target: {targets}')
                # print(f'output: {outputs}')
                # print(f'preds: {preds}')
                # print(f'outputs.sigmoid().cpu(): {outputs.sigmoid().cpu()}')
                # Print accuracy
                print('Accuracy for fold %d: %f %%' % (index, 100.0 * correct / total))
                # print('Accuracy-2 for fold %d: %f %%' % (index, 100.0 * val_corrects / total))
                # print('F1 Score-2 for fold %d: %f ->  %%' %(index, f1_score(targets, preds, zero_division=False)))
                # print('F1 Score-3 for fold %d: %f %%' %(index, f1_score(targets, predicted, zero_division=False)))
                print('F1 Score for fold %d: %f %%' %(index, f1_batch_epoch / total))
                print('Loss for fold %d: %f %%' %(index, val_loss / total))
                # print('F1 Score-5 for fold %d: %f %%' %(index, f1_batch_epoch / test_size))
                # print('F1 Score-6 for fold %d: %f %%' %(index, f1_running / test_size))
            print('----------------------------------------------------------------------------')
            results[index] = [100.0 * (correct / total), 100.0 * f1_batch_epoch / total]
            # results[index][1] = 1


    # ---------------------------- Print fold results ---------------------------- #

    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS')
    print('----------------------------------------------------------------------------"')
    acc_sum = 0.0
    f1_sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: Acc {value[0]}, F1 {value[1]} %')
        acc_sum += value[0]
        f1_sum += value[1]
    print(f'Average: {acc_sum/len(results.items())} %')
    print(f'F1: {f1_sum/len(results.items())} %')

    return results

In [82]:
def epochs_diff(testing_results):
    for loop in testing_results:
        acc_sum = 0.0
        f1_sum = 0.0
        for key, value in loop.items():
            print(f'Fold {key}: Acc {value[0]:.2f}, F1 {value[1]:.2f} %')
            acc_sum += value[0]
            f1_sum += value[1]
        print(f'Average: {acc_sum/len(loop.items()):.2f} %')
        print(f'F1: {f1_sum/len(loop.items()):.2f} %')
        print("-----------------------------")

# 5 FOLD

## Sparse

In [111]:
class sparse_model(nn.Module):
    def __init__(self):
        super(sparse_model, self).__init__()  # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(28, 10),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(10, 4),
            nn.ELU(),
            # nn.Linear(12, 8),
            # nn.ELU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [112]:
model = sparse_model()
dataset = pheme_sparse_final
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(19681.4551), std: tensor(164118.1094)
Variables)
	Train:torch.Size([3723, 1, 28])
	Test: torch.Size([2079, 1, 28])
Train Size 3723 Test Size 2079

Model: sparse_model
Epochs: 100
PATH: ./Model/sparse_model_1.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(129696.0703), std: tensor(1464843.1250)
Variables)
	Train:torch.Size([4659, 1, 28])
	Test: torch.Size([1143, 1, 28])
Train Size 4659 Test Size 1143

Model: sparse_model
Epochs: 100
PATH: ./Model/sparse_model_2.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Training process has finished. Saving trained model.
<Starting TESTING>
------------

In [113]:
epochs_diff(testing_results)

Fold 0: Acc 30.16, F1 20.02 %
Fold 1: Acc 25.11, F1 24.64 %
Fold 2: Acc 51.17, F1 33.10 %
Fold 3: Acc 48.88, F1 32.87 %
Fold 4: Acc 48.08, F1 32.30 %
Average: 40.68 %
F1: 28.59 %
-----------------------------
Fold 0: Acc 48.73, F1 32.45 %
Fold 1: Acc 53.81, F1 34.98 %
Fold 2: Acc 48.61, F1 32.86 %
Fold 3: Acc 50.00, F1 37.78 %
Fold 4: Acc 53.81, F1 34.93 %
Average: 50.99 %
F1: 34.60 %
-----------------------------


In [34]:
# model = sparse_model()
# dataset = pheme_sparse_final
# criterion = nn.BCEWithLogitsLoss()
# epochs = 100

# result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epochs, verbose=)

## POS


In [117]:
class pos_model(nn.Module):
    def __init__(self):
        super(pos_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(21, 4),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [118]:
model = pos_model()
dataset = pheme_pos_final
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(0.8542), std: tensor(1.3381)
Variables)
	Train:torch.Size([3723, 1, 21])
	Test: torch.Size([2079, 1, 21])
Train Size 3723 Test Size 2079

Model: pos_model
Epochs: 200
PATH: ./Model/pos_model_1.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Starting epoch 125
Starting epoch 150
Starting epoch 175
Starting epoch 200
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(0.8214), std: tensor(1.3263)
Variables)
	Train:torch.Size([4659, 1, 21])
	Test: torch.Size([1143, 1, 21])
Train Size 4659 Test Size 1143

Model: pos_model
Epochs: 200
PATH: ./Model/pos_model_2.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Starting epoch 125
Starting epoch 150
St

In [119]:
epochs_diff(testing_results)

Fold 0: Acc 50.12, F1 32.86 %
Fold 1: Acc 51.71, F1 33.57 %
Fold 2: Acc 56.29, F1 35.57 %
Fold 3: Acc 60.11, F1 37.12 %
Fold 4: Acc 55.53, F1 35.47 %
Average: 54.75 %
F1: 34.92 %
-----------------------------


In [120]:
class pos_model(nn.Module):
    def __init__(self):
        super(pos_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(21, 8),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 3),
            nn.ELU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [121]:
model = pos_model()
dataset = pheme_pos_final
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(0.8304), std: tensor(1.3487)
Variables)
	Train:torch.Size([3723, 1, 21])
	Test: torch.Size([2079, 1, 21])
Train Size 3723 Test Size 2079

Model: pos_model
Epochs: 100
PATH: ./Model/pos_model_1.pt



RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x3 and 8x1)

In [ ]:
epochs_diff(testing_results)

## Thread

In [101]:
class thread_model(nn.Module):
    def __init__(self):
        super(thread_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(52, 8),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [102]:
model = thread_model()
dataset = pheme_thread_final_avg
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(2963.2720), std: tensor(24845.1055)
Variables)
	Train:torch.Size([3723, 1, 52])
	Test: torch.Size([2079, 1, 52])
Train Size 3723 Test Size 2079

Model: thread_model
Epochs: 200
PATH: ./Model/thread_model_1.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Starting epoch 125
Starting epoch 150
Starting epoch 175
Starting epoch 200
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(3497.9487), std: tensor(27849.6016)
Variables)
	Train:torch.Size([4659, 1, 52])
	Test: torch.Size([1143, 1, 52])
Train Size 4659 Test Size 1143

Model: thread_model
Epochs: 200
PATH: ./Model/thread_model_2.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Starting epoch

In [103]:
epochs_diff(testing_results)

Fold 0: Acc 64.07, F1 37.88 %
Fold 1: Acc 51.71, F1 33.81 %
Fold 2: Acc 51.81, F1 33.85 %
Fold 3: Acc 51.01, F1 33.25 %
Fold 4: Acc 48.24, F1 31.58 %
Average: 53.37 %
F1: 34.07 %
-----------------------------


In [104]:
class thread_model(nn.Module):
    def __init__(self):
        super(thread_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(52, 12),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(12, 8),
            nn.ELU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [105]:
model = thread_model()
dataset = pheme_thread_final_avg
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(3277.9453), std: tensor(27177.8438)
Variables)
	Train:torch.Size([3723, 1, 52])
	Test: torch.Size([2079, 1, 52])
Train Size 3723 Test Size 2079

Model: thread_model
Epochs: 200
PATH: ./Model/thread_model_1.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Starting epoch 125
Starting epoch 150
Starting epoch 175
Starting epoch 200
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(6453.4585), std: tensor(69573.0312)
Variables)
	Train:torch.Size([4659, 1, 52])
	Test: torch.Size([1143, 1, 52])
Train Size 4659 Test Size 1143

Model: thread_model
Epochs: 200
PATH: ./Model/thread_model_2.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Starting epoch

In [106]:
epochs_diff(testing_results)

Fold 0: Acc 58.78, F1 36.54 %
Fold 1: Acc 50.57, F1 33.10 %
Fold 2: Acc 52.03, F1 34.16 %
Fold 3: Acc 51.57, F1 33.71 %
Fold 4: Acc 54.79, F1 35.05 %
Average: 53.55 %
F1: 34.51 %
-----------------------------


## SPARSE + POS + Thread

In [122]:
# class all_sparse_model(nn.Module):
#     def __init__(self):
#         super(all_sparse_model, self).__init__() # 1*20
#         self.layers = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(52, 8),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         return x

class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 32),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(32, 10),
            nn.ELU(),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [123]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(17072.7910), std: tensor(317982.8125)
Variables)
	Train:torch.Size([3723, 1, 101])
	Test: torch.Size([2079, 1, 101])
Train Size 3723 Test Size 2079

Model: all_sparse_model
Epochs: 50
PATH: ./Model/all_sparse_model_1.pt

Starting epoch 25
Starting epoch 50
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(24228.5430), std: tensor(385957.3750)
Variables)
	Train:torch.Size([4659, 1, 101])
	Test: torch.Size([1143, 1, 101])
Train Size 4659 Test Size 1143

Model: all_sparse_model
Epochs: 50
PATH: ./Model/all_sparse_model_2.pt

Starting epoch 25
Starting epoch 50
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------

In [124]:
epochs_diff(testing_results)

Fold 0: Acc 39.06, F1 26.41 %
Fold 1: Acc 68.77, F1 41.46 %
Fold 2: Acc 46.91, F1 28.64 %
Fold 3: Acc 49.33, F1 35.07 %
Fold 4: Acc 56.76, F1 43.71 %
Average: 52.16 %
F1: 35.06 %
-----------------------------
Fold 0: Acc 76.77, F1 61.71 %
Fold 1: Acc 35.43, F1 24.33 %
Fold 2: Acc 45.63, F1 29.11 %
Fold 3: Acc 58.43, F1 34.96 %
Fold 4: Acc 56.02, F1 34.57 %
Average: 54.46 %
F1: 36.94 %
-----------------------------


In [125]:
# class all_sparse_model(nn.Module):
#     def __init__(self):
#         super(all_sparse_model, self).__init__() # 1*20
#         self.layers = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(52, 8),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         return x

class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 50),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 25),
            nn.ELU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [126]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(2469.1448), std: tensor(24742.1504)
Variables)
	Train:torch.Size([3723, 1, 101])
	Test: torch.Size([2079, 1, 101])
Train Size 3723 Test Size 2079

Model: all_sparse_model
Epochs: 100
PATH: ./Model/all_sparse_model_1.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(16121.2100), std: tensor(338595.2188)
Variables)
	Train:torch.Size([4659, 1, 101])
	Test: torch.Size([1143, 1, 101])
Train Size 4659 Test Size 1143

Model: all_sparse_model
Epochs: 100
PATH: ./Model/all_sparse_model_2.pt

Starting epoch 25
Starting epoch 50
Starting epoch 75
Starting epoch 100
Training process has finished. Saving trained model.
<Starting TESTI

In [127]:
epochs_diff(testing_results)

Fold 0: Acc 49.21, F1 32.42 %
Fold 1: Acc 66.49, F1 39.12 %
Fold 2: Acc 46.06, F1 31.36 %
Fold 3: Acc 56.07, F1 34.48 %
Fold 4: Acc 57.99, F1 36.40 %
Average: 55.16 %
F1: 34.76 %
-----------------------------
Fold 0: Acc 57.53, F1 36.29 %
Fold 1: Acc 58.71, F1 36.36 %
Fold 2: Acc 50.32, F1 32.48 %
Fold 3: Acc 59.66, F1 36.24 %
Fold 4: Acc 60.85, F1 37.88 %
Average: 57.41 %
F1: 35.85 %
-----------------------------


## SPARSE + POS + Thread Standardization

In [150]:
class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 20),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(20, 6),
            nn.ELU(),
            nn.Linear(6, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [151]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [5,10,25,50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False, scaling=True)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(-0.0077), std: tensor(0.8630)
Variables)
	Train:torch.Size([3723, 1, 101])
	Test: torch.Size([2079, 1, 101])
Train Size 3723 Test Size 2079

Model: all_sparse_model
Epochs: 5
PATH: ./Model/all_sparse_model_1.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(-0.0158), std: tensor(0.8134)
Variables)
	Train:torch.Size([4659, 1, 101])
	Test: torch.Size([1143, 1, 101])
Train Size 4659 Test Size 1143

Model: all_sparse_model
Epochs: 5
PATH: ./Model/all_sparse_model_2.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 3
----------------------------------------------------------------------------

In [ ]:
epochs_diff(testing_results)

In [ ]:
# class all_sparse_model(nn.Module):
#     def __init__(self):
#         super(all_sparse_model, self).__init__() # 1*20
#         self.layers = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(52, 8),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         return x

class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 50),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 25),
            nn.ELU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## BERT

In [141]:
class bert_model(nn.Module):
    def __init__(self):
        super(bert_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(768, 50, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 8, bias=True),
            nn.ELU(),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [142]:
model = bert_model()
dataset = pheme_bert_simple_normal
# epochs = 10

testing_epochs = [15, 20]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(0.0401), std: tensor(0.3300)
Variables)
	Train:torch.Size([3723, 1, 768])
	Test: torch.Size([2079, 1, 768])
Train Size 3723 Test Size 2079

Model: bert_model
Epochs: 15
PATH: ./Model/bert_model_1.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(0.0403), std: tensor(0.3283)
Variables)
	Train:torch.Size([4659, 1, 768])
	Test: torch.Size([1143, 1, 768])
Train Size 4659 Test Size 1143

Model: bert_model
Epochs: 15
PATH: ./Model/bert_model_2.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 3
----------------------------------------------------------------------------
mean: tensor(0.0404), s

In [143]:
epochs_diff(testing_results)

Fold 0: Acc 78.84, F1 45.24 %
Fold 1: Acc 74.98, F1 43.35 %
Fold 2: Acc 75.91, F1 44.03 %
Fold 3: Acc 78.54, F1 47.46 %
Fold 4: Acc 73.96, F1 43.38 %
Average: 76.44 %
F1: 44.69 %
-----------------------------
Fold 0: Acc 81.96, F1 46.23 %
Fold 1: Acc 75.59, F1 44.53 %
Fold 2: Acc 69.94, F1 42.24 %
Fold 3: Acc 78.09, F1 47.33 %
Fold 4: Acc 74.20, F1 42.82 %
Average: 75.96 %
F1: 44.63 %
-----------------------------


In [152]:
class bert_model(nn.Module):
    def __init__(self):
        super(bert_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(768, 50, bias=True),
            nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(50, 8, bias=True),
            nn.ELU(),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [153]:
model = bert_model()
# dataset = pheme_bert_simple_normal
dataset = pheme_bert_brackets_normal
# epochs = 10

testing_epochs = [1, 2, 3, 4, 5, 10, 15, 20]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(0.0399), std: tensor(0.3297)
Variables)
	Train:torch.Size([3723, 1, 768])
	Test: torch.Size([2079, 1, 768])
Train Size 3723 Test Size 2079

Model: bert_model
Epochs: 1
PATH: ./Model/bert_model_1.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(0.0399), std: tensor(0.3294)
Variables)
	Train:torch.Size([4659, 1, 768])
	Test: torch.Size([1143, 1, 768])
Train Size 4659 Test Size 1143

Model: bert_model
Epochs: 1
PATH: ./Model/bert_model_2.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 3
----------------------------------------------------------------------------
mean: tensor(0.0399), std

In [154]:
epochs_diff(testing_results)

Fold 0: Acc 67.34, F1 41.44 %
Fold 1: Acc 76.64, F1 47.86 %
Fold 2: Acc 76.76, F1 44.82 %
Fold 3: Acc 80.11, F1 49.84 %
Fold 4: Acc 70.60, F1 42.73 %
Average: 74.29 %
F1: 45.34 %
-----------------------------
Fold 0: Acc 83.65, F1 48.66 %
Fold 1: Acc 76.03, F1 44.23 %
Fold 2: Acc 78.46, F1 45.01 %
Fold 3: Acc 81.80, F1 50.50 %
Fold 4: Acc 74.45, F1 44.29 %
Average: 78.88 %
F1: 46.54 %
-----------------------------
Fold 0: Acc 83.93, F1 49.50 %
Fold 1: Acc 76.82, F1 44.90 %
Fold 2: Acc 77.40, F1 49.54 %
Fold 3: Acc 82.47, F1 53.32 %
Fold 4: Acc 77.72, F1 46.11 %
Average: 79.67 %
F1: 48.67 %
-----------------------------
Fold 0: Acc 83.98, F1 48.03 %
Fold 1: Acc 79.00, F1 45.75 %
Fold 2: Acc 76.12, F1 49.06 %
Fold 3: Acc 82.70, F1 53.39 %
Fold 4: Acc 78.46, F1 48.22 %
Average: 80.05 %
F1: 48.89 %
-----------------------------
Fold 0: Acc 84.80, F1 48.71 %
Fold 1: Acc 79.88, F1 49.44 %
Fold 2: Acc 77.83, F1 48.07 %
Fold 3: Acc 82.25, F1 53.22 %
Fold 4: Acc 78.79, F1 48.27 %
Average: 80.71

## BERT + ALL SPARSE

In [175]:
class bert_sparse_model(nn.Module):
    def __init__(self):
        super(bert_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(869, 72, bias=True),
            nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(72, 10, bias = True),
            nn.ELU(),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [176]:
model = bert_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg, pheme_bert_brackets_normal],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [5,10,50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False,scaling=True)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(-0.0112), std: tensor(0.9256)
Variables)
	Train:torch.Size([3723, 1, 869])
	Test: torch.Size([2079, 1, 869])
Train Size 3723 Test Size 2079

Model: bert_sparse_model
Epochs: 5
PATH: ./Model/bert_sparse_model_1.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(-0.0030), std: tensor(1.0567)
Variables)
	Train:torch.Size([4659, 1, 869])
	Test: torch.Size([1143, 1, 869])
Train Size 4659 Test Size 1143

Model: bert_sparse_model
Epochs: 5
PATH: ./Model/bert_sparse_model_2.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 3
------------------------------------------------------------------------

In [177]:
epochs_diff(testing_results)

Fold 0: Acc 80.28, F1 45.42 %
Fold 1: Acc 79.35, F1 49.16 %
Fold 2: Acc 74.20, F1 49.26 %
Fold 3: Acc 82.58, F1 56.81 %
Fold 4: Acc 80.84, F1 48.34 %
Average: 79.45 %
F1: 49.80 %
-----------------------------
Fold 0: Acc 85.33, F1 50.76 %
Fold 1: Acc 84.95, F1 58.59 %
Fold 2: Acc 84.86, F1 56.13 %
Fold 3: Acc 88.88, F1 59.09 %
Fold 4: Acc 88.29, F1 54.63 %
Average: 86.46 %
F1: 55.84 %
-----------------------------
Fold 0: Acc 88.60, F1 57.05 %
Fold 1: Acc 92.56, F1 79.32 %
Fold 2: Acc 97.44, F1 83.97 %
Fold 3: Acc 99.78, F1 98.14 %
Fold 4: Acc 99.92, F1 99.32 %
Average: 95.66 %
F1: 83.56 %
-----------------------------
Fold 0: Acc 100.00, F1 100.00 %
Fold 1: Acc 100.00, F1 100.00 %
Fold 2: Acc 100.00, F1 100.00 %
Fold 3: Acc 100.00, F1 100.00 %
Fold 4: Acc 100.00, F1 100.00 %
Average: 100.00 %
F1: 100.00 %
-----------------------------


## BERT + ALL SPARSE (STD)

In [ ]:
class bert_sparse_model(nn.Module):
    def __init__(self):
        super(bert_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(869, 72, bias=True),
            nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(72, 10, bias = True),
            nn.ELU(),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [178]:
model = bert_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg, pheme_bert_simple_normal],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [5,10,50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False,scaling=True)
    testing_results.append(result)


FOLD 1
----------------------------------------------------------------------------
mean: tensor(0.0062), std: tensor(0.9920)
Variables)
	Train:torch.Size([3723, 1, 869])
	Test: torch.Size([2079, 1, 869])
Train Size 3723 Test Size 2079

Model: bert_sparse_model
Epochs: 5
PATH: ./Model/bert_sparse_model_1.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 2
----------------------------------------------------------------------------
mean: tensor(0.0021), std: tensor(0.9828)
Variables)
	Train:torch.Size([4659, 1, 869])
	Test: torch.Size([1143, 1, 869])
Train Size 4659 Test Size 1143

Model: bert_sparse_model
Epochs: 5
PATH: ./Model/bert_sparse_model_2.pt

Training process has finished. Saving trained model.
<Starting TESTING>
----------------------------------------------------------------------------
FOLD 3
--------------------------------------------------------------------------

In [179]:
epochs_diff(testing_results)

Fold 0: Acc 81.48, F1 46.14 %
Fold 1: Acc 77.52, F1 46.37 %
Fold 2: Acc 77.40, F1 55.33 %
Fold 3: Acc 81.91, F1 53.08 %
Fold 4: Acc 80.26, F1 50.73 %
Average: 79.71 %
F1: 50.33 %
-----------------------------
Fold 0: Acc 85.81, F1 51.65 %
Fold 1: Acc 83.90, F1 57.59 %
Fold 2: Acc 85.71, F1 53.38 %
Fold 3: Acc 89.21, F1 58.46 %
Fold 4: Acc 86.57, F1 54.06 %
Average: 86.24 %
F1: 55.03 %
-----------------------------
Fold 0: Acc 88.26, F1 57.36 %
Fold 1: Acc 91.25, F1 81.61 %
Fold 2: Acc 93.82, F1 76.13 %
Fold 3: Acc 100.00, F1 100.00 %
Fold 4: Acc 99.92, F1 99.32 %
Average: 94.65 %
F1: 82.89 %
-----------------------------
Fold 0: Acc 100.00, F1 100.00 %
Fold 1: Acc 100.00, F1 100.00 %
Fold 2: Acc 100.00, F1 100.00 %
Fold 3: Acc 100.00, F1 100.00 %
Fold 4: Acc 100.00, F1 100.00 %
Average: 100.00 %
F1: 100.00 %
-----------------------------


# D

In [190]:
print(pheme_event.unique())
print(ext_event.unique())

['charliehebdo' 'ferguson' 'germanwings-crash' 'ottawashooting'
 'sydneysiege']
['ebola-essien' 'prince-toronto' 'putinmissing']


In [191]:
ext_pos_final

,!,#,&,",",@,^,a,d,g,l,n,o,p,r,s,t,u,v,x,z,~
0,0,0,0,0,0,3,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0
1,0,1,2,2,0,1,2,1,0,1,5,2,3,3,0,0,0,6,0,0,0
2,0,0,1,1,0,3,1,4,0,0,4,0,3,0,0,0,0,4,0,0,0
3,0,0,0,2,0,3,2,2,0,0,3,0,1,0,0,0,2,3,0,0,0
4,0,0,0,1,1,3,1,1,0,0,1,0,2,1,0,0,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,0,3,0,0,0,4,0,0,0,0,3,0,3,0,0,0,2,1,0,0,0
481,0,1,0,1,0,2,0,0,0,0,2,0,2,0,0,1,0,2,0,0,0
482,0,1,0,1,0,0,0,1,0,0,2,0,0,0,0,0,1,0,0,0,0
483,0,0,0,2,0,1,2,1,0,0,5,1,2,1,0,0,0,3,0,0,0


In [205]:
print(pheme_sparse_final.shape)
print(pheme_pos_final.shape)

(5802, 28)
(5802, 21)


In [209]:
print(ext_sparse_final.shape)
print(ext_pos_final.shape)
print(ext_bert_simple_normal.shape)
print(ext_bert_brackets_normal.shape)

(623, 28)
(623, 21)
(623, 768)
(623, 768)


In [208]:

# Final
pheme_sparse_final = pd.read_csv('./data/_PHEME_sparse.csv')
pheme_y = pd.read_csv('./data/_PHEME_target.csv').target
pheme_pos_final = pd.read_csv('./data/_PHEME_postags.csv')
pheme_thread_final_avg = pd.read_csv('./data/_PHEME_thread_avg.csv')
pheme_thread_final_std = pd.read_csv('./data/_PHEME_thread_std.csv')

ext_pos_final = pd.read_csv('./data/_PHEMEext_postags.csv')
ext_sparse_final = pd.read_csv('./data/_PHEMEext_sparse.csv')
ext_y = pd.read_csv('./data/_PHEMEext_text.csv').target
ext_thread_final_avg = pd.read_csv('./data/_PHEMEext_thread_avg.csv')
ext_thread_final_std = pd.read_csv('./data/_PHEMEext_thread_std.csv')

pheme_bert_simple_normal = pd.read_csv('./data/_PHEME_Bert_final_simple_nrmzd.csv')
ext_bert_simple_normal = pd.read_csv('./data/_PHEMEext_Bert_final_simple_nrmzd.csv')

pheme_bert_brackets_normal = pd.read_csv('./data/_PHEME_Bert_final_brackets_nrmzd.csv')
ext_bert_brackets_normal = pd.read_csv('./data/_PHEMEext_Bert_final_brackets_nrmzd.csv')

pheme_event = pd.read_csv('./data/_PHEME_text.csv')['Event']
ext_event = pd.read_csv('./data/_PHEMEext_text.csv').Event
pheme_AVGw2v = pd.read_csv('./data/_PHEME_text_AVGw2v.csv').drop(['token'],axis=1)
ext_AVGw2v = pd.read_csv('./data/_PHEMEext_text_AVGw2v.csv').drop(['token'],axis=1)

# 8 Folds

## Sparse

In [ ]:
class sparse_model(nn.Module):
    def __init__(self):
        super(sparse_model, self).__init__()  # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(28, 10),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(10, 4),
            nn.ELU(),
            # nn.Linear(12, 8),
            # nn.ELU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [ ]:
model = sparse_model()
dataset = pheme_sparse_final
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

In [ ]:
# model = sparse_model()
# dataset = pheme_sparse_final
# criterion = nn.BCEWithLogitsLoss()
# epochs = 100

# result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epochs, verbose=)

## POS


In [ ]:
class pos_model(nn.Module):
    def __init__(self):
        super(pos_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(21, 4),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [ ]:
model = pos_model()
dataset = pheme_pos_final
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

In [ ]:
class pos_model(nn.Module):
    def __init__(self):
        super(pos_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(21, 8),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 3),
            nn.ELU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [ ]:
model = pos_model()
dataset = pheme_pos_final
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## Thread

In [ ]:
class thread_model(nn.Module):
    def __init__(self):
        super(thread_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(52, 8),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


In [ ]:
model = thread_model()
dataset = pheme_thread_final_avg
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

In [ ]:
class thread_model(nn.Module):
    def __init__(self):
        super(thread_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(52, 12),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(12, 8),
            nn.ELU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = thread_model()
dataset = pheme_thread_final_avg
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## SPARSE + POS + Thread

In [ ]:
# class all_sparse_model(nn.Module):
#     def __init__(self):
#         super(all_sparse_model, self).__init__() # 1*20
#         self.layers = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(52, 8),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         return x

class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 32),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(32, 10),
            nn.ELU(),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

In [ ]:
# class all_sparse_model(nn.Module):
#     def __init__(self):
#         super(all_sparse_model, self).__init__() # 1*20
#         self.layers = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(52, 8),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         return x

class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 50),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 25),
            nn.ELU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## SPARSE + POS + Thread Standardization

In [ ]:
class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 20),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(20, 6),
            nn.ELU(),
            nn.Linear(6, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [5,10,25,50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False, scaling=True)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

In [ ]:
# class all_sparse_model(nn.Module):
#     def __init__(self):
#         super(all_sparse_model, self).__init__() # 1*20
#         self.layers = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(52, 8),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         return x

class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(101, 50),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 25),
            nn.ELU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = all_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [100,200]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## BERT

In [ ]:
class bert_model(nn.Module):
    def __init__(self):
        super(bert_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(768, 50, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 8, bias=True),
            nn.ELU(),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = bert_model()
dataset = pheme_bert_simple_normal
# epochs = 10

testing_epochs = [15, 20]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

In [ ]:
class bert_model(nn.Module):
    def __init__(self):
        super(bert_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(768, 50, bias=True),
            nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(50, 8, bias=True),
            nn.ELU(),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = bert_model()
# dataset = pheme_bert_simple_normal
dataset = pheme_bert_brackets_normal
# epochs = 10

testing_epochs = [1, 2, 3, 4, 5, 10, 15, 20]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## BERT + ALL SPARSE

In [ ]:
class bert_sparse_model(nn.Module):
    def __init__(self):
        super(bert_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(869, 72, bias=True),
            nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(72, 10, bias = True),
            nn.ELU(),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = bert_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg, pheme_bert_brackets_normal],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [5,10,50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False,scaling=True)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)

## BERT + ALL SPARSE (STD)

In [ ]:
class bert_sparse_model(nn.Module):
    def __init__(self):
        super(bert_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(869, 72, bias=True),
            nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(72, 10, bias = True),
            nn.ELU(),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
model = bert_sparse_model()
dataset = pd.concat([pheme_sparse_final, pheme_pos_final, pheme_thread_final_avg, pheme_bert_simple_normal],axis=1)
criterion = nn.BCEWithLogitsLoss()
# epochs = 10

testing_epochs = [5,10,50,100]
testing_results = []

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epoch, verbose=False,scaling=True)
    testing_results.append(result)

In [ ]:
epochs_diff(testing_results)